In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv


In [2]:
# Import packages and modules
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import timedelta
from keras import backend as K
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.losses import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow import keras
import matplotlib.pyplot as plt

np.random.seed(42)
tf.random.set_seed(42)

In [3]:
df_items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
df_item_categories = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
df_sales_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv', parse_dates=['date'], dayfirst=True)
df_shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
df_test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')  

In [4]:
df_sales_train['date'] = df_sales_train['date'].apply(lambda x: x.strftime('%Y-%m'))
df_train = df_sales_train.pivot_table(index=['shop_id', 'item_id'],
                                      values=['item_cnt_day'], 
                                      columns='date', 
                                      fill_value=0,
                                      aggfunc=np.sum)
df_train = df_train.reset_index()
#df_train = df_train.rename(columns={'item_cnt_day': 'item_cnt_mth'})

df_train.tail()

shop_id item_id item_cnt_day                                          \
date                        2013-01 2013-02 2013-03 2013-04 2013-05 2013-06   
424119      59   22154            1       0       0       0       0       0   
424120      59   22155            0       0       0       0       0       0   
424121      59   22162            0       0       0       0       0       0   
424122      59   22164            0       0       0       0       0       0   
424123      59   22167            0       0       0       0       0       0   

                        ...                                                  \
date   2013-07 2013-08  ... 2015-01 2015-02 2015-03 2015-04 2015-05 2015-06   
424119       0       0  ...       0       0       0       0       0       0   
424120       1       0  ...       0       0       0       0       0       0   
424121       0       0  ...       0       9       4       1       1       0   
424122       0       0  ...       0       2       1       2       0       0   
424123       0       0  ...       0       0       0       0       0       0   

                                        
date   2015-07 2015-08 2015-09 2015-10  
424119       0       0       0       0  
424120       0       0       0       0  
424121       0       1       0       0  
424122       1       0       0       0  
424123       0       0       0       0  

[5 rows x 36 columns]

In [5]:
dataset = df_test.merge(df_train, on=['shop_id', 'item_id'], how='left')
dataset = dataset.fillna(0)
dataset

/opt/conda/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,ID,shop_id,item_id,"(item_cnt_day, 2013-01)","(item_cnt_day, 2013-02)","(item_cnt_day, 2013-03)","(item_cnt_day, 2013-04)","(item_cnt_day, 2013-05)","(item_cnt_day, 2013-06)","(item_cnt_day, 2013-07)",...,"(item_cnt_day, 2015-01)","(item_cnt_day, 2015-02)","(item_cnt_day, 2015-03)","(item_cnt_day, 2015-04)","(item_cnt_day, 2015-05)","(item_cnt_day, 2015-06)","(item_cnt_day, 2015-07)","(item_cnt_day, 2015-08)","(item_cnt_day, 2015-09)","(item_cnt_day, 2015-10)"
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,214196,45,16188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,214197,45,15757,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,214198,45,19648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
dataset = dataset.drop(columns=['shop_id', 'item_id', 'ID'])
dataset

,"(item_cnt_day, 2013-01)","(item_cnt_day, 2013-02)","(item_cnt_day, 2013-03)","(item_cnt_day, 2013-04)","(item_cnt_day, 2013-05)","(item_cnt_day, 2013-06)","(item_cnt_day, 2013-07)","(item_cnt_day, 2013-08)","(item_cnt_day, 2013-09)","(item_cnt_day, 2013-10)",...,"(item_cnt_day, 2015-01)","(item_cnt_day, 2015-02)","(item_cnt_day, 2015-03)","(item_cnt_day, 2015-04)","(item_cnt_day, 2015-05)","(item_cnt_day, 2015-06)","(item_cnt_day, 2015-07)","(item_cnt_day, 2015-08)","(item_cnt_day, 2015-09)","(item_cnt_day, 2015-10)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
X = dataset.values[:, :-1]
y = dataset.values[:, -1:]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, random_state=42)

X_test = dataset.values[:, 1:]
mm_scaler = preprocessing.MinMaxScaler()
X_train = mm_scaler.fit_transform(X_train)
X_valid = mm_scaler.transform(X_valid)
X_test = mm_scaler.transform(X_test)
y_valid = mm_scaler.fit_transform(y_valid)
print(X_train)
print(y_valid)
'''
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]
'''
print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape)

[[0.01169591 0.00847458 0.00384615 ... 0.00207039 0.00228833 0.0021097 ]
 [0.01169591 0.00847458 0.00384615 ... 0.00207039 0.00228833 0.00421941]
 [0.01169591 0.00847458 0.00384615 ... 0.00207039 0.00228833 0.0021097 ]
 ...
 [0.01169591 0.00847458 0.00384615 ... 0.00207039 0.00228833 0.0021097 ]
 [0.01169591 0.00847458 0.00384615 ... 0.00207039 0.00228833 0.0021097 ]
 [0.01169591 0.00847458 0.00384615 ... 0.00207039 0.00228833 0.0021097 ]]
[[0.01941748]
 [0.00970874]
 [0.00970874]
 ...
 [0.00970874]
 [0.00970874]
 [0.00970874]]
(182070, 33) (182070, 1) (32130, 33) (32130, 1) (214200, 33)


In [8]:
'''
from tensorflow.keras import layers 
from tensorflow.keras import metrics

LOSS = 'mse'      
METRICS = ['mae', 'accuracy'] 
OPTIMIZER = tf.keras.optimizers.SGD  
LEARNING_RATE = 0.01  
print(len(X_train))
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[None, 1]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
    ])
model.summary()
model.compile(optimizer=OPTIMIZER(learning_rate=LEARNING_RATE),
              loss=LOSS,
              metrics=METRICS)       

early_stopping = EarlyStopping(
    patience=5, # how many epochs to wait before stopping
    monitor='val_rmse', 
    mode='min',
    restore_best_weights=True,
)

reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_rmse', 
    factor=0.8, 
    patience=2, 
    mode='auto', 
    cooldown=3,
    min_lr=0.00001
)

history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid),
                    callbacks = [early_stopping, reduceLROnPlat])

'''

"\nfrom tensorflow.keras import layers \nfrom tensorflow.keras import metrics\n\nLOSS = 'mse'      \nMETRICS = ['mae', 'accuracy'] \nOPTIMIZER = tf.keras.optimizers.SGD  \nLEARNING_RATE = 0.01  \nprint(len(X_train))\nmodel = keras.Sequential([\n    layers.Dense(64, activation='relu', input_shape=[None, 1]),\n    layers.Dense(64, activation='relu'),\n    layers.Dense(1)\n    ])\nmodel.summary()\nmodel.compile(optimizer=OPTIMIZER(learning_rate=LEARNING_RATE),\n              loss=LOSS,\n              metrics=METRICS)       \n\nearly_stopping = EarlyStopping(\n    patience=5, # how many epochs to wait before stopping\n    monitor='val_rmse', \n    mode='min',\n    restore_best_weights=True,\n)\n\nreduceLROnPlat = ReduceLROnPlateau(\n    monitor='val_rmse', \n    factor=0.8, \n    patience=2, \n    mode='auto', \n    cooldown=3,\n    min_lr=0.00001\n)\n\nhistory = model.fit(X_train, y_train, epochs=5,\n                    validation_data=(X_valid, y_valid),\n                    callbacks = 

In [9]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression()
LR.fit(X_train, y_train)

LinearRegression()

In [10]:
print('Train set mse:', mean_squared_error(y_train, LR.predict(X_train)))
print('Test set mse:', mean_squared_error(y_valid, LR.predict(X_valid)))
print('Test set score:', LR.score(X_train,y_train))
prediction = LR.predict(X_train)
prediction

Train set mse: tf.Tensor(
[1.23448978e-05 3.11726285e-01 5.87783742e-03 ... 5.87783742e-03
 1.33593703e+00 5.87783742e-03], shape=(182070,), dtype=float64)
Test set mse: tf.Tensor(
[1.34189224e+00 4.17701259e-04 8.06961741e-04 ... 7.46077770e-03
 7.46077770e-03 3.00979291e-01], shape=(32130,), dtype=float64)
Test set score: 0.5332848577442153


array([[ 0.00351353],
       [ 0.55832453],
       [-0.07666706],
       ...,
       [-0.07666706],
       [-0.15582742],
       [-0.07666706]])

In [11]:
'''
model = keras.models.Sequential([
    keras.layers.LSTM(30, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(30, return_sequences=True),
    keras.layers.LSTM(30),
    keras.layers.Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), 
              loss = 'mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

early_stopping = EarlyStopping(
    patience=5, # how many epochs to wait before stopping
    monitor='val_rmse', 
    mode='min',
    restore_best_weights=True,
)

reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_rmse', 
    factor=0.8, 
    patience=2, 
    mode='auto', 
    cooldown=3,
    min_lr=0.00001
)

model.summary()

history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid),
                    callbacks = [early_stopping, reduceLROnPlat])

model.save('best_model.h5')
'''

"\nmodel = keras.models.Sequential([\n    keras.layers.LSTM(30, return_sequences=True, input_shape=[None, 1]),\n    keras.layers.LSTM(30, return_sequences=True),\n    keras.layers.LSTM(30),\n    keras.layers.Dense(1)\n])\n\nmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), \n              loss = 'mse',\n              metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])\n\nearly_stopping = EarlyStopping(\n    patience=5, # how many epochs to wait before stopping\n    monitor='val_rmse', \n    mode='min',\n    restore_best_weights=True,\n)\n\nreduceLROnPlat = ReduceLROnPlateau(\n    monitor='val_rmse', \n    factor=0.8, \n    patience=2, \n    mode='auto', \n    cooldown=3,\n    min_lr=0.00001\n)\n\nmodel.summary()\n\nhistory = model.fit(X_train, y_train, epochs=5,\n                    validation_data=(X_valid, y_valid),\n                    callbacks = [early_stopping, reduceLROnPlat])\n\nmodel.save('best_model.h5')\n"

In [12]:
prediction = LR.predict(X_test)

In [13]:
prediction = [float(np.round(x)) for x in prediction]
prediction

[2.0,
 -0.0,
 1.0,
 1.0,
 -0.0,
 -0.0,
 0.0,
 -1.0,
 -1.0,
 -0.0,
 6.0,
 -0.0,
 0.0,
 2.0,
 -0.0,
 -2.0,
 -0.0,
 0.0,
 5.0,
 0.0,
 -1.0,
 -0.0,
 3.0,
 3.0,
 3.0,
 -0.0,
 -0.0,
 5.0,
 -2.0,
 8.0,
 3.0,
 -0.0,
 2.0,
 5.0,
 -0.0,
 -0.0,
 0.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 -1.0,
 4.0,
 14.0,
 0.0,
 -0.0,
 3.0,
 1.0,
 1.0,
 2.0,
 7.0,
 -0.0,
 0.0,
 1.0,
 34.0,
 -1.0,
 6.0,
 8.0,
 -0.0,
 -5.0,
 3.0,
 1.0,
 1.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -2.0,
 -0.0,
 3.0,
 1.0,
 -4.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 2.0,
 -1.0,
 3.0,
 15.0,
 4.0,
 1.0,
 1.0,
 -0.0,
 6.0,
 -0.0,
 27.0,
 12.0,
 4.0,
 1.0,
 7.0,
 1.0,
 -3.0,
 -0.0,
 -0.0,
 0.0,
 1.0,
 1.0,
 -16.0,
 1.0,
 2.0,
 1.0,
 -0.0,
 -0.0,
 -0.0,
 10.0,
 0.0,
 -0.0,
 2.0,
 -0.0,
 -0.0,
 -0.0,
 3.0,
 1.0,
 -0.0,
 0.0,
 24.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 2.0,
 3.0,
 4.0,
 -0.0,
 -0.0,
 -0.0,
 -1.0,
 1.0,
 -0.0,
 1.0,
 5.0,
 -0.0,
 1.0,
 -0.0,
 -0.0,
 3.0,
 -0.0,
 2.0,
 2.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 10.0

In [14]:
df_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
print(df_submission.shape)
df_submission.head()

(214200, 2)


,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [15]:
df_submission['item_cnt_month'] = prediction
df_submission.to_csv('prediction.csv', index=False)
df_submission.head()

,ID,item_cnt_month
0,0,2.0
1,1,-0.0
2,2,1.0
3,3,1.0
4,4,-0.0
